Instrument ID, arm, camera, mediapipe set up

In [1]:
import os 
import sys
import cv2
import numpy as np
import math
import pyrealsense2 as rs
import time
import struct
import socket
import torch
import random
# Import modules from voice-control-instrument-id/voice_instrument_functions.py
#instrument_module_path = os.path.abspath(os.path.join(__file__, "..", "..", "voice-control-instrument-id"))
notebook_path = os.getcwd()  # Current working directory
instrument_module_path = os.path.abspath(os.path.join(notebook_path, "..", "voice-control-instrument-id"))
sys.path.append(instrument_module_path) 
from voice_instrument_functions import *
from audio_utils import *

# Load instrument identification model
inst_model = load_model('../voice-control-instrument-id/models/instrument_detector_model.pt', False)

from robot_control_functions import * # Because of mediapipe, need to import AFTER loading instrument id model

# Set up MyCoBot 280
HOST = "10.42.0.1"
GET_COORDS_PORT = 5006
MOVE_COORDS_PORT = 5005
MOVE_GRIPPER_PORT = 5007
home = [90, 0, 0, -90, 0, -45]
handHome = [[90, 0, 0, 90, 0, -45]]

# Set up Mediapipe hand tracking
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Set up depth camera
# TODO: open as a window for Design Day
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
profile = pipeline.start(config)

#inst_img = get_camera_img(pipeline)

Using cache found in C:\Users\DT12/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-4-21 Python-3.9.21 torch-2.6.0+cpu CPU

Fusing layers... 
Model summary: 267 layers, 46167513 parameters, 0 gradients, 107.8 GFLOPs
Adding AutoShape... 


Voice activation set up - ONLY RUN THIS IF YOU WANT TO SET A CUSTOM VOICE

In [2]:
choose_tts_voice()


Available English TTS Voices:
[0] Microsoft David Desktop - English (United States) (HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0)
[1] Microsoft Zira Desktop - English (United States) (HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0)
Selected voice: Microsoft David Desktop - English (United States)



Main Script

In [ ]:
while True:
    home = [90, 0, 0, -90, 0, -45]
    home[3] = home[3] + random.randint(0, 1)
    
    send_coords(home, HOST, MOVE_COORDS_PORT, 1) # send robot to home
    time.sleep(2) # TODO: better way to wait for arm to be stable before getting frame of tray
    inst_img = get_camera_img(pipeline) # get png of tray to run instrument id

    #command = get_voice_command(porcupine, cobra, recorder) # get voice command
    #inst = get_instrument_name(command) # transcribe voice command and get name of instrument
    #instruments = get_instrument_name()
    #inst = instruments[0][0] ### HARD CODED TO JUST TAKE THE FIRST INSTRUMENT IN THE LIST
    inst = 'scissors'
    x_mid, y_mid = identify_instrument(inst_model, inst_img, inst) # get 2d coords of instrument

    ## Get color and depth frame
    frames = pipeline.poll_for_frames()
    if not frames:
        continue

    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()

    # Convert yolo instrument coords to 3D coords
    inst_coords = get_inst_coords(color_frame, depth_frame, x_mid, y_mid)
    if(np.all(inst_coords == 0)):
        continue

    # target_coords = inst_coords + [home[3], home[4], home[5]]
    coords = get_coords(HOST, GET_COORDS_PORT)
    if coords is None:
            print("Robot did not return coordinates")
            continue
    
    curr_coords = list(coords)
    target_coords = transform_camera_to_robot(inst_coords, curr_coords[:3], curr_coords[3:])
    target_coords[2] = 122
    target_coords = list(target_coords) + [180, 10, 45]

    pickSequence(target_coords, HOST, MOVE_COORDS_PORT, MOVE_GRIPPER_PORT)
    time.sleep(2)

Response from robot: 


C:\Users\DT12/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Box center: (217, 150) | Confidence: 0.31 | Class: Surgical Scissors Sharp/Sharp
Gripper command sent: 100 100
Response from robot: 
Gripper command sent: 0 50
Response from robot: 


C:\Users\DT12/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


No instrument found
Response from robot: 


...

C:\Users\DT12/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


No instrument found
Response from robot: 


C:\Users\DT12/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Box center: (165, 158) | Confidence: 0.44 | Class: Surgical Scissors Sharp/Sharp
Gripper command sent: 100 100
Response from robot: 


KeyboardInterrupt: 